In [277]:
import json
from jinja2 import Environment, PackageLoader, select_autoescape, FileSystemLoader
import datetime
import xml.etree.ElementTree as ET
import ads
import pandas as pd
import bibtexparser

In [224]:
env = Environment(
    loader=FileSystemLoader('templates'),
    block_start_string = '\BLOCK{',
    block_end_string = '}',
    variable_start_string = '\VAR{',
    variable_end_string = '}',
    comment_start_string = '\#{',
    comment_end_string = '}',
    line_statement_prefix = '%%',
    line_comment_prefix = '%#',
    trim_blocks = True,
    autoescape=select_autoescape(['latex'])
)
template_dict = {}

# Read ORCID data #

In [50]:
wkerzendorf_orcid = json.load(open('wkerzendorf_orcid.json'))

In [93]:
prof_preparations = []
for item in wkerzendorf_orcid['activities-summary']['educations']['education-summary'][::-1]:
    cur_entry = {}
    cur_entry['institute'] = item['organization']['name']
    org_address = item['organization']['address']
    if org_address['city'] == 'Heidelberg':
        cur_entry['major'] = 'physics'
    else:
        cur_entry['major'] = 'astrophysics'
    cur_entry['location'] = "{city}, {country}".format(city=org_address['city'], country=org_address['country'])
    cur_entry['degyear'] = "{degree}, {year}".format(degree=item['role-title'], year=item['end-date']['year']['value'])
    prof_preparations.append(cur_entry)
    



In [94]:
for item in wkerzendorf_orcid['activities-summary']['employments']['employment-summary'][::-1]:
    cur_entry = {}
    cur_entry['institute'] = item['organization']['name']
    org_address = item['organization']['address']
    cur_entry['major'] = 'astrophysics'
    cur_entry['location'] = "{city}, {country}".format(city=org_address['city'], country=org_address['country'])
    start_year = item['start-date']['year']['value']
    if item['end-date'] is None:
        end_year = 'present'
    else:
        end_year = item['end-date']['year']['value']
    cur_entry['degyear'] = "{degree}, {start_year} -- {end_year}".format(degree=item['role-title'], 
                                                           start_year=start_year,
                                                           end_year=end_year)
    prof_preparations.append(cur_entry)


In [225]:
template_dict['prof_preparations'] = prof_preparations

# Products #

In [278]:
bparser = bibtexparser.bparser.BibTexParser(interpolate_strings=False)

In [285]:
bibtex_db = bibtexparser.load(open('wekerzendorf_all_added.bib'), bparser)

In [286]:
related_products_bcodes = ['2018MNRAS.477.3425B',
 '2017arXiv171100055K',
 '2017Natur.551...75S',
 '2014MNRAS.440..387K',
 '2017A&A...599A..46B']
general_products_bcodes = ['2009ApJ...701.1665K',
 '2014ApJ...796L..26K',
 '2013A&A...558A..33A',
 '2012ApJ...759....7K',
 '2017MNRAS.472.2534K']

In [312]:
def shorten_authors(author_str):
    author_list = author_str.replace('\n', ' ').split(' and ')
    if len(author_list) < 8:
        return author_str
    
        

In [ ]:
x.entries_dict = bibtex_db

In [302]:
all_products_dict = {}
for bcode in related_products_bcodes + general_products_bcodes:
    all_products_dict[bcode] = bibtex_db.entries_dict[bcode]

In [314]:
!open -a bibdesk wekerzendorf_all_added.bib

In [304]:
all_products_db = bibtexparser.bibdatabase.BibDatabase()
all_products_db.entries = all_products_dict.values()

In [324]:
with open('formats/nsf_biosketch/wekerzendorf_2018.bib', 'w') as bibtex_file:
    bibtexparser.dump(all_products_db, bibtex_file)

In [326]:
!open -a bibdesk formats/nsf_biosketch/wekerzendorf_2018.bib

# Synergistics #

In [205]:
ads_key = json.load(open('keys.json'))['ads_key']
ads.config.token = ads_key

In [232]:
astropy_stats = {}

In [207]:
cur_year = datetime.datetime.now().year
astropy_ads_query_str = 'topn({topn}, year:{start_year}-{end_year} database:astronomy property:refereed, citation_count desc)'
astropy_query = ads.SearchQuery(q=astropy_ads_query_str.format(topn=20, start_year=cur_year-5, end_year=cur_year+1), rows=1000)
papers = {}
for paper in astropy_query:
    papers[paper.bibcode] = paper
print(astropy_query.response.get_ratelimits())

{'limit': '5000', 'remaining': '4999', 'reset': '1541646791'}


In [213]:
df_paper = pd.DataFrame(index=papers.keys(), columns=['citations'])
for i, paper in enumerate(papers.values()):
    df_paper.loc[paper.bibcode, 'citations'] = paper.citation_count
    

/Users/wkerzend/miniconda/envs/cv/lib/python3.7/site-packages/ads/utils.py:31: UserWarning: You are lazy loading attributes via 'citation_count', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [233]:
astropy_stats['citation_rank_last5'] = int(20 - df_paper.citations.rank()['2013A&A...558A..33A']) + 1

In [230]:
astropy_oh = ET.parse('astropy.xml').getroot()

In [245]:
astropy_stats['contributors'] = astropy_oh.find('./result/project/analysis/total_contributor_count').text
astropy_stats['code_lines'] = astropy_oh.find('./result/project/analysis/total_code_lines').text

In [246]:
template_dict['astropy_stats'] = astropy_stats

In [247]:
tardis_stats = {}

In [248]:
tardis_oh = ET.parse('tardis.xml')
tardis_root = tardis_oh.getroot()

In [249]:
tardis_stats['contributors'] = tardis_root.find('./result/project/analysis/total_contributor_count').text
tardis_stats['code_lines'] = tardis_root.find('./result/project/analysis/total_code_lines').text

In [250]:
template_dict['tardis_stats'] = tardis_stats

# Writing the template #

In [327]:
nsf_template = env.get_template('nsf_biosketch_template.tex')

In [328]:
with open('formats/nsf_biosketch/nsf_biosketch.tex', 'w') as fh:
    fh.write(nsf_template.render(**template_dict))

In [329]:
!code formats/nsf_biosketch/nsf_biosketch.tex